In [27]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START, END
from langgraph.graph.message import add_messages





In [28]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model = "llama3.2:1b", temperature=0)

In [29]:
class chatstate(TypedDict):
    question: Annotated[list, add_messages]
    answer: Annotated[list, add_messages]
    easy: int
    medium: int
    hard: int
    curr_question_type: str
    eval_score: int
    question_count: int
    final_report: str


In [30]:
def question_bot(language, field, DSA, difficulty="easy"):
    prompt = f"You are a teacher with expertise in {language}, {field}, and {DSA}. Ask a {difficulty} level question."
    return {"questions": [llm.invoke(prompt)]}


In [31]:
def choose_question(curr_question_type, eval_score, question_count):
    if curr_question_type == "easy" and eval_score >= 5:
        new_type = "medium"
    elif curr_question_type == "medium":
        if eval_score >= 8:
            new_type = "hard"
        elif eval_score < 5:
            new_type = "easy"
        else:
            new_type = "medium"
    elif curr_question_type == "hard" and eval_score < 7:
        new_type = "medium"
    else:
        new_type = curr_question_type

    question_count += 1
    return {
        "curr_question_type": new_type,
        "question_count": question_count
    }


In [32]:
def should_continue(question_count):
    if question_count < 15:
        return "next_question"
    else:
        return "generate_report"


In [33]:
def generate_report(state):
    # Summarize user's performance
    prompt = f"Based on these answers and scores, write a short evaluation highlighting strengths, weaknesses, and areas for improvement."
    combined_data = f"Answers: {state['answer']}, Eval Scores: {state['eval_score']}"
    report = llm.invoke(prompt + combined_data)
    return {"final_report": report}


In [34]:
def evaluate(state):
    answer = state["answer"][-1]  # Get the latest answer
    question_type = state["curr_question_type"]
    eval_score = state["eval_score"]

    # For simplicity, let's simulate the evaluation as a correctness check
    # You can later plug in a tool/LLM to verify answer quality more deeply
    if "correct" in answer.lower():  # Simulated evaluation rule
        eval_score += 1
        if question_type == "easy":
            state["easy"] += 1
        elif question_type == "medium":
            state["medium"] += 1
        elif question_type == "hard":
            state["hard"] += 1
    else:
        eval_score += 0  # No change if wrong (can also subtract if desired)

    return {
        "eval_score": eval_score,
        "easy": state["easy"],
        "medium": state["medium"],
        "hard": state["hard"]
    }


In [ ]:
builder = StateGraph(chatstate)

# 1. Register ALL nodes
builder.add_node("questioning", question_bot)
builder.add_node("evaluate", evaluate)
builder.add_node("choosing question", choose_question)
builder.add_node("continue?", should_continue)
builder.add_node("generate_report", generate_report)

# 2. Set entry point
builder.set_entry_point("questioning")

# 3. Add edges
builder.add_edge("questioning", "evaluate")
builder.add_edge("evaluate", "choosing question")
builder.add_edge("choosing question", "continue?")

# ✅ Now: Conditional edges from a registered node to other registered nodes
builder.add_conditional_edges("continue?", {
    "next_question": "questioning",
    "generate_report": "generate_report"
})


TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>